In [1]:
import argparse
from pathlib import Path
import numpy as np
import wandb

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast

torch.cuda.empty_cache()

# Utils

In [2]:
DIR_BASE = Path('/home/daniel/sphinxsys-build/cases_jan/heart/bin/data')


def dir_path_validator(value):
    p = Path(value)
    if not p.exists() or not p.is_dir():
        msg = "{value} is not valid path to an existing directory".format(
            value=value)
        raise argparse.ArgumentTypeError(msg)
    return p.resolve()


MODEL_SIZES = {
    'S': (32, 64, 128),
    'M': (64, 128, 256),
    'L': (128, 256, 512)
}


# Model Definition

In [3]:
class Autoencoder(nn.Module):
    def __init__(self, time_step=3, filter_sizes=(32, 64, 128)):
        super(Autoencoder, self).__init__()

        self.encoder_layers_0 = self._encoder_layer(3*time_step, filter_sizes[0])
        self.encoder_layers_1 = self._encoder_layer(filter_sizes[0], filter_sizes[1])
        self.encoder_layers_2 = self._encoder_layer(filter_sizes[1], filter_sizes[2])

        self.bottleneck_layers = nn.Sequential(
            nn.Conv3d(filter_sizes[2], filter_sizes[2]*2, 3, padding='same'),
            nn.BatchNorm3d(filter_sizes[2]*2),
            nn.LeakyReLU(),
        )

        self.decoder_layers_0 = self._decoder_layer(filter_sizes[2]*2, filter_sizes[2])
        self.decoder_layers_1 = self._decoder_layer(filter_sizes[2], filter_sizes[1])
        self.decoder_layers_2 = self._decoder_layer(filter_sizes[1], filter_sizes[0])
        
        self.output_layers = nn.Sequential(
            nn.Conv3d(filter_sizes[0], 1, 3, padding='same'),
            nn.BatchNorm3d(1),
            nn.Tanh(),
        )

    def _encoder_layer(self, in_channels, out_channels):
        layers = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, 3, padding='same'),
            nn.BatchNorm3d(out_channels),
            nn.LeakyReLU(),
            nn.MaxPool3d(2),
        )
        return layers

    def _decoder_layer(self, in_channels, out_channels):
        layers = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv3d(in_channels, out_channels, 3, padding='same'),
            nn.BatchNorm3d(out_channels),
            nn.LeakyReLU(),
        )
        return layers

    def forward(self, x):
        out = self.encoder_layers_0(x)
        out = self.encoder_layers_1(out)
        out = self.encoder_layers_2(out)
        out = self.bottleneck_layers(out)
        out = self.decoder_layers_0(out)
        out = self.decoder_layers_1(out)
        out = self.decoder_layers_2(out)
        out = self.output_layers(out)
        return out

class SPHDataset(Dataset):
    def __init__(self, data_dir):
        self.data_files = list(data_dir.glob('*.npz'))
    
    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        file = np.load(self.data_files[idx])
        X, Y = file['Displacement'], file['Voltage']
        X = torch.from_numpy(X)
        Y = torch.from_numpy(Y)
        return X, Y

# 

# Main

In [4]:
# Set up wandb
wandb.init(project="SPH-CNN", entity="ddeng00")
wandb.config = {
    'epochs': 25,
    'batch_size': 14,
    'model_size': 'S',
    'time_step': 3,
    'learning_rate': 0.001,
    # 'test_split': 0.05,
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ddeng00 (use `wandb login --relogin` to force relogin)


In [5]:
# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [6]:
# Load data
sph_dataset = SPHDataset(DIR_BASE)
sph_dataloader = DataLoader(sph_dataset, batch_size=wandb.config['batch_size'], num_workers=4, shuffle=True, pin_memory=True)


# test_size = int(wandb.config['test_split'] * len(sph_dataset))
# train_size = len(sph_dataset) - test_size
# train_dataset, test_dataset = random_split(
#     sph_dataset, [train_size, test_size])
# train_dataloader = DataLoader(
#     train_dataset, batch_size=wandb.config['batch_size'], shuffle=True, pin_memory=True)
# test_dataloader = DataLoader(
#     test_dataset, batch_size=wandb.config['batch_size'], shuffle=True)


In [7]:
# Instantiate Model
model = Autoencoder(time_step=wandb.config['time_step'],
                    filter_sizes=MODEL_SIZES[wandb.config['model_size']]).to(device, dtype=torch.float)

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=wandb.config['learning_rate'])

## Training

In [8]:
scaler = GradScaler()
for epoch in range(wandb.config['epochs']):
    train_loss = 0.0
    for input, target in sph_dataloader:
        input = input.to(device, dtype=torch.float, non_blocking=True)
        target = target.to(device, dtype=torch.float, non_blocking=True)
        optimizer.zero_grad()
        with autocast():
            output = model(input)
            loss = criterion(output, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()

    train_loss = train_loss / len(sph_dataloader)
    wandb.log({'train_loss': train_loss})

In [10]:
torch.save(model.state_dict(), Path('/home/daniel/sphinxsys-build/cases_jan/heart/bin/model.pt'))